In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import glob
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import grad
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.utils import save_image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [2]:
# Root directory for project
proj_root = "/datasets/home/32/232/tdobhal/Project/"

# Root directory for dataset
data_root = "/datasets/home/32/232/tdobhal/Project/6_train/images/"

# Number of images in the directory
num_images = 23418

# Batch size during training
batch_size = 16

# Spatial size of training images. All images will be resized to this size using a transformer.
image_size = 256

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 100

# Number of training loops for critic/discriminator
num_critic = 2

# Learning rate for optimizers
lr = 0.00005

# Alpha hyperparam for RMS optimizers
alpha = 0.9

# Lambda hyperparam for gradient penalty
lambda_gradient = 0.1

# Number of GPUs available. Use 0 for CPU mode.
ngpu = torch.cuda.device_count()

# Device to run on
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
class DataLoader:
    def __init__(self):
        '''
        Parameters:
        
        '''
        self.device = device
        self.data_path = data_root
        self.image_size = image_size
        self.batch_size = batch_size
        self.train_names = glob.glob(self.data_path + 'real_A/*')
        self.names = [self.train_names[i].split('/')[-1] for i in range(len(self.train_names))]
        self.data_transforms = torchvision.transforms.Compose([
                torchvision.transforms.Resize(image_size),
                torchvision.transforms.CenterCrop(image_size),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
    
    def image_loader(self, image_name):
        """load image, returns cuda tensor"""
        image = Image.open(image_name)
        image = self.data_transforms(image).float()
        image = torch.autograd.Variable(image, requires_grad=False)
        image = image.unsqueeze(0)  # this is for VGG, may not be needed for ResNet
        return image[0].to(self.device)  # assumes that you're using GPU

    def show(self, img):
        npimg = img.cpu().detach().numpy()
        npimg = np.transpose(npimg, (1,2,0))
        if npimg.shape[2] == 3:
            plt.imshow(npimg)
        else:
            plt.imshow(npimg[:,:,0], cmap='gray')
            
    def imshow(self, img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.cpu().detach().numpy()
        plt.figure(figsize = (10,2))
        plt.imshow(np.transpose(npimg, (1, 2, 0)), aspect='auto')

    def data_generator(self):
        root = self.data_path
        batch_size = self.batch_size
        
        images_dir = root + 'real_A/'
        labels_dir = root + 'fake_B/'

        while True:
            x, y = [], []
            idx = np.random.choice(self.names, batch_size)
            for i in range(idx.shape[0]):
                x.append(self.image_loader(images_dir + idx[i]))
                y.append(self.image_loader(labels_dir + idx[i]))
            yield torch.stack(x), torch.stack(y)

In [4]:
data = DataLoader()

In [5]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.01)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.01)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        
        # Convolution layers
        
        # input is (nc) x 256 x 256
        self.conv1 = nn.Conv2d(nc, ngf, 4, 2, 1, bias=True)
        self.lr1 = nn.LeakyReLU(inplace=True)
        # state size. (ngf) x 128 x 128
        self.conv2 = nn.Conv2d(ngf, ngf*2, 4, 2, 1, bias=True)
        self.bn2 = nn.BatchNorm2d(ngf*2)
        self.lr2 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*2) x 64 x 64
        self.conv3 = nn.Conv2d(ngf*2, ngf*4, 4, 2, 1, bias=True)
        self.bn3 = nn.BatchNorm2d(ngf*4)
        self.lr3 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*4) x 32 x 32
        self.conv4 = nn.Conv2d(ngf*4, ngf*8, 4, 2, 1, bias=True)
        self.bn4 = nn.BatchNorm2d(ngf*8)
        self.lr4 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 16 x 16
        self.conv5 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn5 = nn.BatchNorm2d(ngf*8)
        self.lr5 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 8 x 8
        self.conv6 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn6 = nn.BatchNorm2d(ngf*8)
        self.lr6 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 4 x 4
        self.conv7 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn7 = nn.BatchNorm2d(ngf*8)
        self.lr7 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 2 x 2
        self.conv8 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn8 = nn.BatchNorm2d(ngf*8)
        self.r8 = nn.ReLU(inplace=True)
        
        # Transpose Convolutional Layers
        
        # input is (ngf*8) x 1 x 1
        self.tr_conv1 = nn.ConvTranspose2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn1 = nn.BatchNorm2d(ngf*8)
        self.tr_r1 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 2 x 2
        self.tr_conv2 = nn.ConvTranspose2d((ngf*8)*2, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn2 = nn.BatchNorm2d(ngf*8)
        self.tr_r2 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 4 x 4
        self.tr_conv3 = nn.ConvTranspose2d((ngf*8)*2, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn3 = nn.BatchNorm2d(ngf*8)
        self.tr_r3 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 8 x 8
        self.tr_conv4 = nn.ConvTranspose2d((ngf*8)*2, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn4 = nn.BatchNorm2d(ngf*8)
        self.tr_r4 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 16 x 16
        self.tr_conv5 = nn.ConvTranspose2d((ngf*8)*2, ngf*4, 4, 2, 1, bias=True)
        self.tr_bn5 = nn.BatchNorm2d(ngf*4)
        self.tr_r5 = nn.ReLU(inplace=True)
        # state size. (ngf*4)*2 x 32 x 32
        self.tr_conv6 = nn.ConvTranspose2d((ngf*4)*2, ngf*2, 4, 2, 1, bias=True)
        self.tr_bn6 = nn.BatchNorm2d(ngf*2)
        self.tr_r6 = nn.ReLU(inplace=True)
        # state size. (ngf*2)*2 x 64 x 64
        self.tr_conv7 = nn.ConvTranspose2d((ngf*2)*2, ngf, 4, 2, 1, bias=True)
        self.tr_bn7 = nn.BatchNorm2d(ngf)
        self.tr_r7 = nn.ReLU(inplace=True)
        # state size. (ngf)*2 x 128 x 128
        self.tr_conv8 = nn.ConvTranspose2d((ngf)*2, nc, 4, 2, 1, bias=True)
        self.out = nn.Tanh()
        # state size. (nc) x 256 x 256
    
    def forward(self, x):
        c1 = self.conv1(x)
        c2 = self.bn2(self.conv2(self.lr1(c1)))
        c3 = self.bn3(self.conv3(self.lr2(c2)))
        c4 = self.bn4(self.conv4(self.lr3(c3)))
        c5 = self.bn5(self.conv5(self.lr4(c4)))
        c6 = self.bn6(self.conv6(self.lr5(c5)))
        c7 = self.bn7(self.conv7(self.lr6(c6)))
        c8 = self.bn8(self.conv8(self.lr7(c7)))
        
        t1 = self.tr_bn1(self.tr_conv1(self.r8(c8)))
        t1 = torch.cat((t1, c7), dim=1)
        t2 = self.tr_bn2(self.tr_conv2(self.tr_r1(t1)))
        t2 = torch.cat((t2, c6), dim=1)
        t3 = self.tr_bn3(self.tr_conv3(self.tr_r2(t2)))
        t3 = torch.cat((t3, c5), dim=1)
        t4 = self.tr_bn4(self.tr_conv4(self.tr_r3(t3)))
        t4 = torch.cat((t4, c4), dim=1)
        t5 = self.tr_bn5(self.tr_conv5(self.tr_r4(t4)))
        t5 = torch.cat((t5, c3), dim=1)
        t6 = self.tr_bn6(self.tr_conv6(self.tr_r5(t5)))
        t6 = torch.cat((t6, c2), dim=1)
        t7 = self.tr_bn7(self.tr_conv7(self.tr_r6(t6)))
        t7 = torch.cat((t7, c1), dim=1)
        t8 = self.tr_conv8(self.tr_r7(t7))
        t8 = self.out(t8)
        return t8
    

In [7]:
gen_a = Generator().to(device)
gen_b = Generator().to(device)

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            # input is (nc) x 256 x 256
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 128 x 128
            nn.MaxPool2d((2, 2)), 
            
            # state size. (ndf) x 64 x 64
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 32 x 32
            nn.MaxPool2d((2, 2)),
    
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, ndf * 8, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            )
        
        self.flat = nn.Linear(ndf * 8 * 2 * 2, 1)
        
    def forward(self, x):
        x = self.net(x)
        x = x.view(x.size()[0], -1)
        x = self.flat(x)
        return x

In [9]:
dis_a = Discriminator().to(device)
dis_b = Discriminator().to(device)

In [10]:
class EpochTracker():
    def __init__(self, in_file):
        self.epoch = 0
        self.iter = 0
        self.in_file = in_file
        self.file_exists = os.path.isfile(in_file)
        if self.file_exists:
            with open(in_file, 'r') as f: 
                d = f.read() 
                a, b = d.split(";")
                self.epoch = int(a)
                self.iter = int(b)
     
    def write(self, epoch, iteration):
        self.epoch = epoch
        self.iter = iteration
        data = "{};{}".format(self.epoch, self.iter)
        with open(self.in_file, 'w') as f:
            f.write(data)

In [11]:
# DataParallel for more than 1 gpu
# gen_a = nn.DataParallel(gen_a, list(range(ngpu)))
# dis_a = nn.DataParallel(dis_a, list(range(ngpu)))
# gen_b = nn.DataParallel(gen_b, list(range(ngpu)))
# dis_b = nn.DataParallel(dis_b, list(range(ngpu)))

gen_a.apply(weights_init_normal)
dis_a.apply(weights_init_normal)
gen_b.apply(weights_init_normal)
dis_b.apply(weights_init_normal)

Discriminator(
  (net): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): LeakyReLU(negative_slope=0.2, inplace)
    (10): Conv2d(512, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (11): LeakyReLU(negative_slope=0.2, inplace)
  )
  (flat): Linear(in_features=2048, out_features=1, bias=True)
)

In [12]:
# Gradient penalty for Wasserstein Loss
def gradient_penalty(real_dis, fake_dis, discriminator):
    shape = [real_dis.size(0)] + [1] * (real_dis.dim() - 1)
    alpha = torch.rand(shape).to(device)
    z = real_dis + alpha * (fake_dis - real_dis)

    # gradient penalty
    z = Variable(z, requires_grad=True).to(device)
    output = discriminator(z)
    g = grad(output, z, grad_outputs=torch.ones(output.size()).to(device), create_graph=True)[0].view(z.size(0), -1)
    gp = ((g.norm(p=2, dim=1) - 1)**2).mean()

    return lambda_gradient * gp

In [2]:
criterion_pixel = nn.L1Loss()

optim_gen_a = torch.optim.RMSprop(gen_a.parameters(), lr, alpha)
optim_gen_b = torch.optim.RMSprop(gen_b.parameters(), lr, alpha)
optim_dis_a = torch.optim.RMSprop(dis_a.parameters(), lr, alpha)
optim_dis_b = torch.optim.RMSprop(dis_b.parameters(), lr, alpha)

NameError: name 'gen_a' is not defined

### Train Loop

In [ ]:
sample_interval = 25
checkpoint_interval = 100
file_prefix = proj_root + 'saved_models/dual_wgans2/'

e_tracker = EpochTracker(file_prefix + 'epoch.txt')

if(e_tracker.file_exists):
    gen_a.load_state_dict(torch.load(file_prefix + 'generator_a.pth'))
    dis_a.load_state_dict(torch.load(file_prefix + 'discriminator_a.pth'))
    gen_b.load_state_dict(torch.load(file_prefix + 'generator_b.pth'))
    dis_b.load_state_dict(torch.load(file_prefix + 'discriminator_b.pth'))

    
for epoch in range(e_tracker.epoch, num_epochs):
    for i in range(num_images // batch_size):
        if epoch == e_tracker.epoch and i <= e_tracker.iter:
            continue    
        
        for j in range(num_critic):
            x, y = next(data.data_generator())
            real_a = Variable(x).to(device)
            real_b = Variable(y).to(device)
        
            # Training Discriminator A with real_A batch
            optim_dis_a.zero_grad();
            pred_real_dis_a = dis_a(real_a).view(-1)
            
            # Training Discriminator B with real_B batch
            optim_dis_b.zero_grad();
            pred_real_dis_b = dis_b(real_b).view(-1)
            
            # Training Discriminator B with fake_B batch of Generator A
            fake_b = gen_a(real_a)
            pred_fake_dis_b = dis_b(fake_b.detach()).view(-1)
            
            # Training Discriminator A with fake_A batch of Generator B
            fake_a = gen_b(real_b)
            pred_fake_dis_a = dis_a(fake_a.detach()).view(-1)
            
            # Update params of Discriminator A and B
            err_dis_a = pred_fake_dis_a.mean() - pred_real_dis_a.mean() + gradient_penalty(real_a.data, fake_a.data, dis_a)
            err_dis_a.backward()
            optim_dis_a.step()
            err_dis_b = pred_fake_dis_b.mean() - pred_real_dis_b.mean() + gradient_penalty(real_b.data, fake_b.data, dis_b)
            err_dis_b.backward()
            optim_dis_b.step()
            
        # Train and update Generator A based on Discriminator B's prediction
        x, y = next(data.data_generator())
        real_a = Variable(x).to(device)
        real_b = Variable(y).to(device)
        
        optim_gen_a.zero_grad()
        fake_b_gen = gen_a(fake_a)
        pred_out_dis_b = dis_b(fake_b_gen).view(-1)
        err_gen_a_pred = -pred_out_dis_b.mean()
        err_gen_a_pixel = criterion_pixel(fake_b_gen, real_b)
        err_gen_a = err_gen_a_pred + err_gen_a_pixel
        err_gen_a.backward()
        optim_gen_a.step()
        
        # Train and update Generator B based on Discriminator A's prediction
        optim_gen_b.zero_grad()
        fake_a_gen = gen_b(fake_b)
        pred_out_dis_a = dis_a(fake_a_gen).view(-1)
        err_gen_b_pred = -pred_out_dis_a.mean()
        err_gen_b_pixel = criterion_pixel(fake_a_gen, real_a)
        err_gen_b = err_gen_b_pred + err_gen_b_pixel
        err_gen_b.backward()
        optim_gen_b.step()
        
        # Print statistics and save checkpoints
        print("\r[Epoch %d/%d] [Batch %d/%d] [D_A loss: %f] [D_B loss: %f] [G_A loss: %f, G_B loss: %f]" %
                                                        (epoch, num_epochs,
                                                        i, num_images//batch_size,
                                                        err_dis_a.item(), err_dis_b.item(), 
                                                        err_gen_a.item(), err_gen_b.item()))

        if i % sample_interval == 0:
            img_sample = torch.cat((real_a.data, fake_a.data, real_b.data, fake_b.data), -2)
            save_image(img_sample, proj_root + 'saved_images/dual_wgans2/%d_%d.png' % (epoch, i), nrow=5, normalize=True)

            torch.save(gen_a.state_dict(), proj_root + 'saved_models/dual_wgans2/generator_a.pth')
            torch.save(gen_b.state_dict(), proj_root + 'saved_models/dual_wgans2/generator_b.pth')
            torch.save(dis_a.state_dict(), proj_root + 'saved_models/dual_wgans2/discriminator_a.pth')
            torch.save(dis_b.state_dict(), proj_root + 'saved_models/dual_wgans2/discriminator_b.pth')
            e_tracker.write(epoch, i)

[Epoch 0/100] [Batch 376/1463] [D_A loss: 3.624161] [D_B loss: -0.582017] [G_A loss: 17.132257, G_B loss: 221.347229]
[Epoch 0/100] [Batch 377/1463] [D_A loss: 2.647049] [D_B loss: 1.298769] [G_A loss: 25.230881, G_B loss: 175.617249]
[Epoch 0/100] [Batch 378/1463] [D_A loss: -3.374094] [D_B loss: -0.396629] [G_A loss: 30.199238, G_B loss: -68.910294]
[Epoch 0/100] [Batch 379/1463] [D_A loss: -1.817593] [D_B loss: -1.088959] [G_A loss: 40.297436, G_B loss: 4.145512]
[Epoch 0/100] [Batch 380/1463] [D_A loss: -0.783661] [D_B loss: 0.596369] [G_A loss: 35.619064, G_B loss: -131.685089]
[Epoch 0/100] [Batch 381/1463] [D_A loss: 5.895774] [D_B loss: 0.741749] [G_A loss: 30.809788, G_B loss: 92.037659]
[Epoch 0/100] [Batch 382/1463] [D_A loss: -2.385512] [D_B loss: -0.034267] [G_A loss: 25.034981, G_B loss: 278.535614]
[Epoch 0/100] [Batch 383/1463] [D_A loss: 2.875027] [D_B loss: -0.179871] [G_A loss: 22.917915, G_B loss: 302.011444]
[Epoch 0/100] [Batch 384/1463] [D_A loss: 1.139215] [D_B 

[Epoch 0/100] [Batch 446/1463] [D_A loss: 0.503611] [D_B loss: -0.154540] [G_A loss: 13.466799, G_B loss: 54.934639]
[Epoch 0/100] [Batch 447/1463] [D_A loss: -0.200527] [D_B loss: -0.009726] [G_A loss: 12.458622, G_B loss: 24.497839]
[Epoch 0/100] [Batch 448/1463] [D_A loss: 0.321373] [D_B loss: 0.110224] [G_A loss: 11.490746, G_B loss: 7.014593]
[Epoch 0/100] [Batch 449/1463] [D_A loss: 0.202745] [D_B loss: 0.033442] [G_A loss: 14.412921, G_B loss: 164.982269]
[Epoch 0/100] [Batch 450/1463] [D_A loss: 0.526280] [D_B loss: -0.107980] [G_A loss: 15.332762, G_B loss: 114.682892]
[Epoch 0/100] [Batch 451/1463] [D_A loss: -1.394155] [D_B loss: 0.089080] [G_A loss: 14.386254, G_B loss: 54.547249]
[Epoch 0/100] [Batch 452/1463] [D_A loss: 0.616868] [D_B loss: -0.002745] [G_A loss: 15.211162, G_B loss: 118.713043]
[Epoch 0/100] [Batch 453/1463] [D_A loss: -1.821199] [D_B loss: 0.043251] [G_A loss: 12.852713, G_B loss: -11.475356]
[Epoch 0/100] [Batch 454/1463] [D_A loss: 1.126154] [D_B loss:

[Epoch 0/100] [Batch 516/1463] [D_A loss: -2.575162] [D_B loss: 0.146241] [G_A loss: 19.583708, G_B loss: -30.820450]
[Epoch 0/100] [Batch 517/1463] [D_A loss: 0.012522] [D_B loss: 0.003594] [G_A loss: 18.958775, G_B loss: -31.477276]
[Epoch 0/100] [Batch 518/1463] [D_A loss: 1.432787] [D_B loss: -0.314750] [G_A loss: 18.968697, G_B loss: 110.232407]
[Epoch 0/100] [Batch 519/1463] [D_A loss: 0.894515] [D_B loss: 0.064844] [G_A loss: 14.516290, G_B loss: 92.924713]
[Epoch 0/100] [Batch 520/1463] [D_A loss: -0.630604] [D_B loss: 0.161998] [G_A loss: 13.492724, G_B loss: 106.794937]
[Epoch 0/100] [Batch 521/1463] [D_A loss: -0.586298] [D_B loss: 0.039946] [G_A loss: 14.758097, G_B loss: 53.585590]
[Epoch 0/100] [Batch 522/1463] [D_A loss: -0.481173] [D_B loss: 0.015903] [G_A loss: 15.713711, G_B loss: 83.122734]
[Epoch 0/100] [Batch 523/1463] [D_A loss: -0.321624] [D_B loss: -0.286005] [G_A loss: 16.143497, G_B loss: 165.616180]
[Epoch 0/100] [Batch 524/1463] [D_A loss: 0.338962] [D_B los

[Epoch 0/100] [Batch 586/1463] [D_A loss: -0.281707] [D_B loss: 0.152818] [G_A loss: 26.331938, G_B loss: 58.083889]
[Epoch 0/100] [Batch 587/1463] [D_A loss: 1.045324] [D_B loss: -0.220862] [G_A loss: 29.854132, G_B loss: 187.083023]
[Epoch 0/100] [Batch 588/1463] [D_A loss: 1.825454] [D_B loss: -0.316573] [G_A loss: 32.931007, G_B loss: 221.122284]
[Epoch 0/100] [Batch 589/1463] [D_A loss: 0.012722] [D_B loss: -0.055780] [G_A loss: 35.517136, G_B loss: 211.335663]
[Epoch 0/100] [Batch 590/1463] [D_A loss: -1.083031] [D_B loss: 0.127826] [G_A loss: 36.017410, G_B loss: 69.858803]
[Epoch 0/100] [Batch 591/1463] [D_A loss: 0.546446] [D_B loss: 0.036183] [G_A loss: 33.254395, G_B loss: 17.580729]
[Epoch 0/100] [Batch 592/1463] [D_A loss: -0.821432] [D_B loss: -0.434144] [G_A loss: 35.066277, G_B loss: 102.263824]
[Epoch 0/100] [Batch 593/1463] [D_A loss: -1.607527] [D_B loss: 0.337021] [G_A loss: 37.161888, G_B loss: 120.783379]
[Epoch 0/100] [Batch 594/1463] [D_A loss: -1.465072] [D_B l

[Epoch 0/100] [Batch 656/1463] [D_A loss: -3.406855] [D_B loss: 0.132232] [G_A loss: 32.854378, G_B loss: -8.060356]
[Epoch 0/100] [Batch 657/1463] [D_A loss: 0.402509] [D_B loss: -0.435495] [G_A loss: 29.746902, G_B loss: 24.061064]
[Epoch 0/100] [Batch 658/1463] [D_A loss: -0.226442] [D_B loss: -0.079502] [G_A loss: 28.297287, G_B loss: 131.506378]
[Epoch 0/100] [Batch 659/1463] [D_A loss: -2.982185] [D_B loss: -0.037243] [G_A loss: 28.868032, G_B loss: 227.379044]
[Epoch 0/100] [Batch 660/1463] [D_A loss: -2.922874] [D_B loss: -0.108698] [G_A loss: 35.913544, G_B loss: 243.680573]
[Epoch 0/100] [Batch 661/1463] [D_A loss: 1.154917] [D_B loss: -0.309115] [G_A loss: 40.707642, G_B loss: 184.676315]
[Epoch 0/100] [Batch 662/1463] [D_A loss: 7.708685] [D_B loss: 0.010305] [G_A loss: 41.109730, G_B loss: 83.033501]
[Epoch 0/100] [Batch 663/1463] [D_A loss: 0.500891] [D_B loss: -0.252395] [G_A loss: 46.929070, G_B loss: 60.631561]
[Epoch 0/100] [Batch 664/1463] [D_A loss: -0.543774] [D_B 